Código para la creación de imagen tiff con metadatos de drone

In [51]:
from PIL import Image
from PIL.TiffTags import TAGS
import re
import rasterio

# %Crear Dicciónario de metadata de las imágenes %

In [52]:
# Nombre de la imagen

imagen = "./data/fotos/DJI_0051.TIF"

n_imagen = imagen.split("/")[-1]
n_imagen = n_imagen.split(".")[0]
n_imagen


'DJI_0051'

In [53]:
# obtenemos la imagen .tif

img = Image.open(imagen)
# obtenemos los Tags de la metadata y se almacenan en un diccionario
meta_dict = {TAGS[key]: img.tag[key] for key in img.tag_v2}
# Se imprime el diccionario para obtener la composición de los datos
# for rec in meta_dict:
#     print(rec, ":", meta_dict[rec])

# Extraemos el indicador xmp
p = meta_dict.get("XMP")
s = p.decode("UTF-8")
# dividimos por el salto de linea y obtenemos una lista
div = s.split("\n")

#eliminamos los espacios vacios 

for ind,recorrido in enumerate(div):
    div[ind]=recorrido.strip() 
usar = div[17]

result = re.search(":(.*)=", div[17])
result.group(1)

result2 = re.search("\"(.*)\"", div[17])
result2.group(1)

metadiccionario = {}
for ind,recorrido in enumerate(div):
    try:
        metadiccionario[re.search(":(.*)=", div[ind]).group(1)] = re.search("\"(.*)\"", div[ind]).group(1)
    except:
        pass

metadiccionario = {}
for ind,recorrido in enumerate(div):
    try:
        metadiccionario[re.search(":(.*)=", div[ind]).group(1)] = re.search("\"(.*)\"", div[ind]).group(1)
    except:
        pass

print(metadiccionario)

{'xmpmeta xmlns:x': 'adobe:ns:meta/', 'RDF xmlns:rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#', 'Description rdf:about': 'DJI Meta Data', 'tiff': 'http://ns.adobe.com/tiff/1.0/', 'exif': 'http://ns.adobe.com/exif/1.0/', 'xmp': 'http://ns.adobe.com/xap/1.0/', 'xmpMM': 'http://ns.adobe.com/xap/1.0/mm/', 'dc': 'http://purl.org/dc/elements/1.1/', 'crs': 'http://ns.adobe.com/camera-raw-settings/1.0/', 'drone-dji': 'http://www.dji.com/drone-dji/1.0/', 'Camera': 'http://pix4d.com/camera/1.0', 'ModifyDate': '2022-02-11', 'CreateDate': '2022-02-11', 'Make': 'DJI', 'Model': 'FC6360', 'format': 'image/TIF', 'AbsoluteAltitude': '+84.65', 'RelativeAltitude': '+70.03', 'GpsLatitude': '11.22617803', 'GpsLongitude': '-74.18549349', 'GimbalRollDegree': '+0.00', 'GimbalYawDegree': '-96.50', 'GimbalPitchDegree': '-89.90', 'FlightRollDegree': '+6.40', 'FlightYawDegree': '-95.30', 'FlightPitchDegree': '+6.50', 'FlightXSpeed': '+0.10', 'FlightYSpeed': '+0.40', 'FlightZSpeed': '+0.00', 'CamReverse': '0

## *Extracción de los datos significativos de la metadata*

In [54]:

centro_x = float(metadiccionario["CalibratedOpticalCenterX"])
centro_y = float(metadiccionario["CalibratedOpticalCenterY"])
altura_vuelo = float(metadiccionario["RelativeAltitude"])
distancia_focal = float(metadiccionario["CalibratedFocalLength"])
angulo_giro = float(metadiccionario["GimbalYawDegree"])
latitud = float(metadiccionario["GpsLatitude"])
longitud = float(metadiccionario["GpsLongitude"])

#print(float(metadiccionario["GimbalYawDegree"]))

#Se calcula la resolución en metros por pixel.
#Es decir, a cuantos metros equivale un pixel
resolucion = altura_vuelo/distancia_focal
print(f"Resolución = {resolucion}")

#rotado=img.rotate(math.degrees(float(metadiccionario["GimbalYawDegree"])))



Resolución = 0.03660104451823564


### Se utiliza la rotación para crear nuevas imágenes y se abren en otra variable(Temporal)
Proyectividad: Reemplaza la misma imágen de entrada con la rotación correspondiente para que la imágen quede geoposicionadamente con el mismo nombre

Image.rotate(angle, resample=Resampling.NEAREST, expand=0, center=None, translate=None, fillcolor=None)[source]

- Angle= angulo
- Expand = Hace la imágen mas grande para que cuando gira no se corta
- fillCollor = Rellena con cualquier color la función

In [55]:
# Utilizamos en angulo que se usa
rotado = img.rotate(
    (float(metadiccionario["GimbalYawDegree"]))*-1, expand=True)
# Guardamos con el nombre el nuevo .TIF con el imágen rotada
rotado.save("data/rotado_"+n_imagen+".tif")
imagen2 = "data/rotado_"+n_imagen+".tif"


In [56]:
# Abrimos ahora la segunda imágen, la cual es la nueva creada
with rasterio.open(imagen2) as src:
    # Obtener la información de la imagen
    width = src.width
    height = src.height
    transform = src.transform
    crs = src.crs
    print(src.res)
    print("width:", width)
    print(src.transform)


(1.0, 1.0)
width: 1474
| 1.00, 0.00, 0.00|
| 0.00, 1.00, 0.00|
| 0.00, 0.00, 1.00|


Cálculo de las coordenadas de las esquinas la imagen

Se divide entre 111111 para convertir los metros en grados. Esto se debe a que 1 grado de longitud o latitud en el ecuador es aproximadamente 111.111 km de ancho. Dado que la longitud y la latitud son medidas en grados, se debe convertir el tamaño de los píxeles en metros a grados para poder calcular las coordenadas extremas en grados.
Donde el ancho de la imagen en metros se divide entre la distancia en metros por grado para obtener la diferencia en grados. El factor 111111 es una aproximación, ya que la distancia en metros por grado varía según la latitud. Sin embargo, esta aproximación es suficiente para una precisión razonable.

Es importante tener en cuenta que esta conversión solo es válida para latitudes cercanas al ecuador, ya que la distancia en metros por grado varía según la latitud.

In [57]:
# Se geoposiciona la imágen y ahora
# Calcular las coordenadas de los extremos en metros
min_lon = float(metadiccionario["GpsLongitude"]) - (
    float(metadiccionario["CalibratedOpticalCenterX"]) * resolucion) / 111111
max_lon = float(metadiccionario["GpsLongitude"]) + (
    float(metadiccionario["CalibratedOpticalCenterX"]) * resolucion) / 111111
min_lat = float(metadiccionario["GpsLatitude"]) - (
    float(metadiccionario["CalibratedOpticalCenterY"]) * resolucion) / 111111
max_lat = float(metadiccionario["GpsLatitude"]) + (
    float(metadiccionario["CalibratedOpticalCenterY"]) * resolucion) / 111111
print(min_lon, max_lon, min_lat, max_lat)

transform = rasterio.transform.from_bounds(
    min_lon, min_lat, max_lon, max_lat, img.width, img.height)
""" print(transform)
print("#"*10) """
# Abrir el archivo original
with rasterio.open(imagen2) as src:
    # Leer los datos de la imagen
    image = src.read()

    # Crear una copia del archivo
    with rasterio.open(
        "data/drone_rot_"+n_imagen+".TIF",  # Nombre del nuevo archivo georefenciado
        'w',
        driver='GTiff',
        height=src.height,
        width=src.width,
        count=1,
        dtype="float64",
        crs={"init": "epsg:4326"},
        transform=transform,
    ) as dst:
        dst.write(image)


-74.18575701778406 -74.18522996221594 11.225963913675452 11.226392146324548
